# Compilation Examples with Routing

In [5]:

from pytket import Circuit
from pytket.extensions.aqt.backends.aqt_multi_zone import AQTMultiZoneBackend
from pytket.extensions.aqt.multi_zone_architecture.circuit_routing.routing_config import (
    RoutingConfig,
)
from pytket.extensions.aqt.multi_zone_architecture.compilation_settings import (
    CompilationSettings,
)
from pytket.extensions.aqt.multi_zone_architecture.graph_algs.mt_kahypar import (
    MtKahyparConfig,
    configure_mtkahypar,
)
from pytket.extensions.aqt.multi_zone_architecture.initial_placement.settings import (
    InitialPlacementAlg,
    InitialPlacementSettings,
)
from pytket.extensions.aqt.multi_zone_architecture.trap_architecture.named_architectures import (
    four_zones_in_a_line,
    grid12,
    grid12_mod,
    racetrack,
    racetrack_4_gatezones,
)
from pytket.extensions.aqt.multi_zone_architecture.circuit_routing.gate_selection import (
    GraphPartitionGateSelector,
    HypergraphPartitionGateSelector,
)

First lets define some circuits

In [6]:
## A simple GHZ circuit
ghz_circuit = Circuit(16)
ghz_circuit.H(0)
for i in range(ghz_circuit.n_qubits - 1):
    ghz_circuit.CX(i, i + 1)
ghz_circuit.measure_all()

## Quantum advantage circuit template
def quantum_advantage_circuit(Lx: int, Ly: int) -> Circuit:
    N = Lx * Ly
    Ntrot = 30
    dt = 0.2
    hx = 2
    J = 1

    def c(x, y):
        if (x - y) % N == Lx or (y - x) % N == Lx:
            return 1
        if x // Lx == y // Lx and x == y - (y % Lx) + (((y % Lx) + 1) % Lx):
            return 1
        if x // Lx == y // Lx and y == x - (x % Lx) + (((x % Lx) + 1) % Lx):
            return 1
        return 0

    coupling_list = []
    for i in range(N):
        for j in range(i + 1, N):
            if c(i, j) == 1:
                coupling_list.append([i, j])

    U = Circuit(N)

    for t in range(Ntrot):
        n_trotter_steps = t

        if n_trotter_steps == 0:
            for j in range(N):
                U.Rx(-2 * dt * hx / 2, j)

        for coupling in coupling_list:
            U.ZZPhase(-2 * dt * J, coupling[0], coupling[1])
        for j in range(N):
            U.Rx(-2 * dt * hx, j)

        if n_trotter_steps == Ntrot - 1:
            for j in range(N):
                U.Rx(2 * dt * hx / 2, j)
    return U

## 56 qubit quantum advantage circuit
advantage_circuit_56 = quantum_advantage_circuit(7, 8)

## 30 qubit quantum advantage circuit
advantage_circuit_30 = quantum_advantage_circuit(5, 6)

def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
    return circ

## 16 qft circuit
qft_circ = build_qft_circuit(16)



Now some MultiZone Backends

In [7]:
line_backend = AQTMultiZoneBackend(
    architecture=four_zones_in_a_line, access_token="invalid"
)

racetrack_backend = AQTMultiZoneBackend(
    architecture=racetrack, access_token="invalid"
)

racetrack_4_gatezones_backend = AQTMultiZoneBackend(
    architecture=racetrack_4_gatezones, access_token="invalid"
)

grid_backend = AQTMultiZoneBackend(
    architecture=grid12, access_token="invalid"
)

grid_mod_backend = AQTMultiZoneBackend(
    architecture=grid12_mod, access_token="invalid"
)


And some different compilation settings

In [8]:
# This can be used to configure the number of threads used and random seed
# for mt-kahypar. It is not required (then default will be used) and can only
# be set once
configure_mtkahypar(
    MtKahyparConfig(
        n_threads = 1,
        random_seed = 13
    )
)
order_init = InitialPlacementSettings(
    algorithm=InitialPlacementAlg.qubit_order,
    zone_free_space=2,
    max_depth=200,
)

graph_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=RoutingConfig(gate_selector=GraphPartitionGateSelector()),
)

hypergraph_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=RoutingConfig(gate_selector=HypergraphPartitionGateSelector()),
)

greedy_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=RoutingConfig(),
)
legacy_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=RoutingConfig(use_legacy_greedy_method=True),
)

## GHZ circuit on a line architecture with two central gate zones and two outer memory zones (8-6-6-8)

In [9]:
import logging
from pytket.extensions.aqt.logger import configure_logging
ghz_precompiled = line_backend.compile_circuit(ghz_circuit, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
configure_logging(level = logging.DEBUG)
ghz_graph_routed = line_backend.route_compiled(ghz_precompiled, graph_compilation_settings)
ghz_hypergraph_routed = line_backend.route_compiled(ghz_precompiled, hypergraph_compilation_settings)
configure_logging(level = logging.INFO)
ghz_greedy_routed = line_backend.route_compiled(ghz_precompiled, greedy_compilation_settings)
ghz_greedy_routed_legacy = line_backend.route_compiled(ghz_precompiled, legacy_compilation_settings)
print("Shuttles, swaps graph: ", ghz_graph_routed.get_n_shuttles(), ", ", ghz_graph_routed.get_n_pswaps())
print("Shuttles, swaps hypergraph: ", ghz_hypergraph_routed.get_n_shuttles(), ", ", ghz_hypergraph_routed.get_n_pswaps())
print("Shuttles, swaps greedy: ", ghz_greedy_routed.get_n_shuttles(), ", ", ghz_greedy_routed.get_n_pswaps())
print("Shuttles, swaps greedy legacy: ", ghz_greedy_routed_legacy.get_n_shuttles(), ", ", ghz_greedy_routed_legacy.get_n_pswaps())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
2025-12-02 16:10:26 [DEBUG] gate_selection.graph_partition_gate_selection: --- Depth List ---


2025-12-02 16:10:26 [DEBUG] gate_selection.graph_partition_gate_selection: 0: gate_pairs [(0, 1)], blocks [{0, 1}] 
2025-12-02 16:10:26 [DEBUG] gate_selection.graph_partition_gate_selection: 1: gate_pairs [(1, 2)], blocks [{0, 1, 2}] 
2025-12-02 16:10:26 [DEBUG] gate_selection.graph_partition_gate_selection: 2: gate_pairs [(2, 3)], blocks [{0, 1, 2, 3}] 
2025-12-02 16:10:26 [DEBUG] gate_selection.graph_partition_gate_selection: 3: gate_pairs [(3, 4)], blocks [{0, 1, 2, 3, 4}] 
2025-12-02 16:10:26 [DEBUG] graph_algs.mt_kahypar: cut_cost: 700000
2025-12-02 16:10:26 [DEBUG] circuit_routing.route_circuit: --- Configuration change 0 ---
2025-12-02 16:10:26 [DEBUG] circuit_routing.route_circuit: Z0: [0, 1, 2, 3, 4, 5] -> [6, 7] -- (+6, +7, -0, -1, -2, -3, -4, -5)
2025-12-02 16:10:26 [DEBUG] circuit_routing.route_circuit: Z1: [6, 7, 8, 9] -> [0, 1, 2, 3, 4, 5] -- (+0, +1, +2, +3, +4, +5, -8, -9, -6, -7)
2025-12-02 16:10:26 [DEBUG] circuit_routing.route_circuit: Z2: [10, 11, 12, 13] -> [8, 9, 

## QFT circuit on a line architecture with two central gate zones and two outer memory zones (8-6-6-8)

In [10]:
configure_logging(level = logging.WARNING)
qft_precompiled = line_backend.compile_circuit(qft_circ, graph_compilation_settings)
print("#1qb gates =", qft_precompiled.n_1qb_gates())
print("#gate depth =", qft_precompiled.depth())
print("#2qb gates =", qft_precompiled.n_2qb_gates())
print("#2qb gate depth =", qft_precompiled.depth_2q())
qft_graph_routed = line_backend.route_compiled(qft_precompiled, graph_compilation_settings)
qft_hypergraph_routed = line_backend.route_compiled(qft_precompiled, hypergraph_compilation_settings)
qft_greedy_routed = line_backend.route_compiled(qft_precompiled, greedy_compilation_settings)
qft_greedy_routed_legacy = line_backend.route_compiled(qft_precompiled, legacy_compilation_settings)
print("Shuttles, swaps graph: ", qft_graph_routed.get_n_shuttles(), ", ", qft_graph_routed.get_n_pswaps())
print("Shuttles, swaps hypergraph: ", qft_hypergraph_routed.get_n_shuttles(), ", ", qft_hypergraph_routed.get_n_pswaps())
print("Shuttles, swaps greedy: ", qft_greedy_routed.get_n_shuttles(), ", ", qft_greedy_routed.get_n_pswaps())
print("Shuttles, swaps greedy legacy: ", qft_greedy_routed_legacy.get_n_shuttles(), ", ", qft_greedy_routed_legacy.get_n_pswaps())

#1qb gates = 569
#gate depth = 155
#2qb gates = 264
#2qb gate depth = 61
2025-12-02 16:11:25 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:11:25 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:11:25 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:11:25 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
Shuttles, swaps graph:  79 ,  274
Shuttles, swaps hypergraph:  58 ,  167
Shuttles, swaps greedy:  91 ,  256
Shuttles, swaps greedy legacy:  312 ,  934


## 56 qubit "quantum advantage" circuit on all gate-zone "racetrack" architecture

In [ ]:

adv_precomp = racetrack_backend.compile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =",  adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
for name, settings in [
    ("graph", graph_compilation_settings),
    ("hypergraph", hypergraph_compilation_settings),
    ("greedy", greedy_compilation_settings),
    ("greedy legacy",  legacy_compilation_settings)]:
    adv_routed = racetrack_backend.route_compiled(adv_precomp, settings)
    print(f"Shuttles, swaps {name}: " , adv_routed.get_n_shuttles(),", ", adv_routed.get_n_pswaps() )

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
2025-12-02 16:15:44 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:15:46 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:15:47 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:15:52 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:15:53 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:16:00 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using

## 56 qubit "quantum advantage" circuit on 4 sequential gate-zone "racetrack" architecture

In [14]:

adv_precomp2 = racetrack_4_gatezones_backend.compile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp2.n_1qb_gates())
print("#gate depth =", adv_precomp2.depth())
print("#2qb gates =", adv_precomp2.n_2qb_gates())
print("#2qb gate depth =", adv_precomp2.depth_2q())
for name, settings in [
    ("graph", graph_compilation_settings),
    ("hypergraph", hypergraph_compilation_settings),
    ("greedy", greedy_compilation_settings),
    ("greedy legacy",  legacy_compilation_settings)]:
    adv_routed = racetrack_backend.route_compiled(adv_precomp2, settings)
    print(f"Shuttles, swaps {name}: " , adv_routed.get_n_shuttles(),", ", adv_routed.get_n_pswaps() )

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
2025-12-02 16:21:36 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:21:44 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:21:50 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:21:50 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:21:51 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
2025-12-02 16:21:51 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using

## Grid Architecture: All gate zones

In [12]:

adv_precomp = grid_backend.compile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
for name, settings in [
    ("graph", graph_compilation_settings),
    ("hypergraph", hypergraph_compilation_settings),
    ("greedy", greedy_compilation_settings),
    ("greedy legacy",  legacy_compilation_settings)]:
    adv_routed = racetrack_backend.route_compiled(adv_precomp, settings)
    print(f"Shuttles, swaps {name}: " , adv_routed.get_n_shuttles(),", ", adv_routed.get_n_pswaps() )

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
2025-12-02 16:18:18 [WARNING] circuit_routing.route_circuit: Chosen gate selector did not produce new configuration. Using greedy gate selection for this round
Shuttles, swaps graph:  6365 ,  8943
Shuttles, swaps hypergraph:  10692 ,  10160
Shuttles, swaps greedy:  5158 ,  11294
Shuttles, swaps greedy legacy:  4974 ,  15156


## Grid Architecture: Four gate zones

In [13]:
adv_gm_precompiled = grid_mod_backend.compile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_gm_precompiled.n_1qb_gates())
print("#gate depth =", adv_gm_precompiled.depth())
print("#2qb gates =", adv_gm_precompiled.n_2qb_gates())
print("#2qb gate depth =", adv_gm_precompiled.depth_2q())
for name, settings in [
    ("graph", graph_compilation_settings),
    ("hypergraph", hypergraph_compilation_settings),
    ("greedy", greedy_compilation_settings),
    ("greedy legacy",  legacy_compilation_settings)]:
    adv_routed = racetrack_backend.route_compiled(adv_gm_precompiled, settings)
    print(f"Shuttles, swaps {name}: " , adv_routed.get_n_shuttles(),", ", adv_routed.get_n_pswaps() )

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
Shuttles, swaps graph:  7097 ,  7449
Shuttles, swaps hypergraph:  10372 ,  10219
Shuttles, swaps greedy:  5158 ,  11294
Shuttles, swaps greedy legacy:  4974 ,  15156
